----
# 1. Module
----

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.2 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm

import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

---
# 2. Data
---

In [3]:
path = '/content/drive/MyDrive/2. Study/DACON/data/train.csv'

data = pd.read_csv(path)
data

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


---
# 3. Preprocessing
----

- Max_length 확인


In [4]:
total_data_text = list(data['문장'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  63.8257058218971
문장길이 최대 :  534
문장길이 표준편차 :  35.47661313872456
pad_sequences maxlen :  134
전체 문장의 0.9565322531890454%가 maxlen 설정값 이내에 포함됩니다. 


- 다중 출력 모델을 만들기 위해 타겟값 변경

In [5]:
trans = {'유형' : {'사실형' : 0, '추론형' : 1, '예측형' : 2, '대화형' : 3},
         '극성' : {'긍정' : 0, '부정' : 1, '미정' : 2},
         '시제' : {'현재': 0, '과거' : 1, '미래' : 2},
         '확실성' : {'확실' : 0, '불확실' : 1}
         }

In [6]:
data['유형'] = data['유형'].apply(lambda x : trans['유형'][x])
data['극성'] = data['극성'].apply(lambda x : trans['극성'][x])
data['시제'] = data['시제'].apply(lambda x : trans['시제'][x])
data['확실성'] = data['확실성'].apply(lambda x : trans['확실성'][x])

In [7]:
data

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,0,0,0,0,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,0,0,1,0,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,0,0,2,0,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",0,0,1,0,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,0,0,0,0,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",0,0,1,0,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",0,0,1,0,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,0,0,1,0,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,0,0,1,1,사실형-긍정-과거-불확실


- 문장 전처리

In [8]:
def extract_word(text):
  convert = re.compile('^\d*\d$|[^가-힣a-zA-Z0-9.%ㅋㅜㅎㅠ]') # 해당 문자만 남기고 나머지 제거
  result = convert.sub(' ',text) # 변환

  result = re.sub(r'[" "]+', " ",result) # 공백 여러개를 한개로 변환
  result = result.strip() # 양쪽 공백 제거

  return result

In [9]:
import random
extract_word(data['문장'][random.randint(0, len(data))])

'잡배들이 떼를 지어 욕을 하고 옷을 잡아당기는 등 욕을 보이는 자까지 있으니 이는 큰 악풍이다.'

In [10]:
data['문장'] = data['문장'].apply(lambda x : extract_word(x))

In [11]:
data

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,0,0,0,0,사실형-긍정-현재-확실
1,TRAIN_00001,이어 앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정 이라며 그 이전이라도...,0,0,1,0,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,0,0,2,0,사실형-긍정-미래-확실
3,TRAIN_00003,서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만 하루 만에 차...,0,0,1,0,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,0,0,0,0,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,신동덤 은 신비한 동물사전 과 해리 포터 시리즈를 잇는 마법 어드벤처물로 전편에 이...,0,0,1,0,사실형-긍정-과거-확실
16537,TRAIN_16537,수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목 어깨 팔꿈치 등 허...,0,0,1,0,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 고...,0,0,1,0,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,0,0,1,1,사실형-긍정-과거-불확실


In [12]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
model = AutoModel.from_pretrained('klue/roberta-large')

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [13]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (d

In [14]:
print(tokenizer.encode(data['문장'][2]))

[0, 3659, 2116, 6870, 2116, 4509, 2069, 3627, 27, 2429, 3797, 6071, 2299, 2118, 14160, 2103, 5426, 1863, 2069, 3740, 9, 3604, 5319, 9, 3610, 3989, 4538, 18, 2]


In [15]:
token = tokenizer(
        data['문장'][1], return_tensors = 'pt',
        truncation = True, max_length = 134,
        padding = 'max_length', add_special_tokens = True
                            )

In [16]:
print(token['input_ids'].shape)
print(token['token_type_ids'].shape)
print(token['attention_mask'].shape)

torch.Size([1, 134])
torch.Size([1, 134])
torch.Size([1, 134])


---
# 4. Config
----

In [18]:
class Config():
  max_length = 134
  model = 'klue/bert-base'
  drop_rate = 0.3
  batch_size = 32
  warmup_ratio = 0.2
  lr = 1e-5
  num_workers = 8
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  epoch_n = 10
  max_grad_norm = 1
  save_path = '/content/drive/MyDrive/2. Study/DACON'
  submission = '/content/drive/MyDrive/2. Study/DACON/data/sample_submission.csv'

if __name__ == '__main__':
  args = Config()  

---
# 5. Dataset & Dataloader
---

In [19]:
class custom_dataset(Dataset):
  def __init__(self, data, max_len):
    self.data = data
    self.max_len = max_len
    self.tokenizer = AutoTokenizer.from_pretrained(args.model)

    self.label_1 = [np.int64(i) for i in data['유형']]
    self.label_2 = [np.int64(i) for i in data['극성']]
    self.label_3 = [np.int64(i) for i in data['시제']]
    self.label_4 = [np.int64(i) for i in data['확실성']]

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    
    data = self.data.loc[idx]
    input = data['문장']
    
    inputs = self.tokenizer(
        input, return_tensors = 'pt',
        truncation = True, max_length = self.max_len,
        padding = 'max_length', add_special_tokens = True
                            )

    input_ids = inputs['input_ids'].reshape(-1)
    token_type_ids = inputs['token_type_ids'].reshape(-1)
    attention_mask = inputs['attention_mask'].reshape(-1)

    return input_ids, token_type_ids, attention_mask, self.label_1[idx], self.label_2[idx], self.label_3[idx], self.label_4[idx]

In [20]:
data_train = custom_dataset(data, args.max_length)
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, 
                                               batch_size=args.batch_size, 
                                               num_workers=args.num_workers, drop_last=True)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [21]:
for a in zip(train_dataloader):
  print(a)
  break

([tensor([[   2, 3890, 2178,  ...,    0,    0,    0],
        [   2, 1919, 4936,  ...,    0,    0,    0],
        [   2, 3995, 3931,  ...,    0,    0,    0],
        ...,
        [   2, 1144, 4121,  ...,    0,    0,    0],
        [   2, 3696, 4374,  ...,    0,    0,    0],
        [   2, 4619, 2259,  ...,    0,    0,    0]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), tenso

---
# 6. Model
---

In [22]:
class BERTClassifier(nn.Module):
    def __init__(self, hidden_size = 768,
                 dr_rate=None, params=None,):
        super(BERTClassifier, self).__init__()

        self.model = AutoModel.from_pretrained(args.model)
        self.dr_rate = dr_rate

        self.label_1 = nn.Linear(hidden_size , 4)
        self.label_2 = nn.Linear(hidden_size , 3)
        self.label_3 = nn.Linear(hidden_size , 3)
        self.label_4 = nn.Linear(hidden_size , 2)

        self.dropout = nn.Dropout(p=dr_rate)

        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask, token_type_ids):

        out = self.model(input_ids = input_ids.long(),
                         token_type_ids = token_type_ids.long(),
                         attention_mask = attention_mask.to(args.device))
        
        out = self.dropout(out.pooler_output)

        label_1_out = self.label_1(out)
        label_2_out = self.label_2(out)
        label_3_out = self.label_3(out)
        label_4_out = self.label_4(out)

        return self.sigmoid(label_1_out), self.sigmoid(label_2_out), self.sigmoid(label_3_out), self.sigmoid(label_4_out)

---
# 7. Loss
---

In [24]:
class FocalLoss(nn.Module):
    def __init__(self,
                 alpha  = None,
                 gamma = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
 
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v!r}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x, y):
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss   

- [ASL 설명 블로그](https://yphy.tistory.com/20)

In [25]:
class ASLSingleLabel(nn.Module):
    '''
    This loss is intended for single-label classification problems
    '''
    def __init__(self, gamma_pos=0, gamma_neg=4, eps: float = 0.1, reduction='mean'):
        super(ASLSingleLabel, self).__init__()

        self.eps = eps
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        self.targets_classes = []
        self.gamma_pos = gamma_pos
        self.gamma_neg = gamma_neg
        self.reduction = reduction

    def forward(self, inputs, target):
        '''
        "input" dimensions: - (batch_size,number_classes)
        "target" dimensions: - (batch_size)
        '''
        num_classes = inputs.size()[-1]
        log_preds = self.logsoftmax(inputs)
        self.targets_classes = torch.zeros_like(inputs).scatter_(1, target.long().unsqueeze(1), 1)

        # ASL weights
        targets = self.targets_classes
        anti_targets = 1 - targets
        xs_pos = torch.exp(log_preds)
        xs_neg = 1 - xs_pos
        xs_pos = xs_pos * targets
        xs_neg = xs_neg * anti_targets
        asymmetric_w = torch.pow(1 - xs_pos - xs_neg,
                                 self.gamma_pos * targets + self.gamma_neg * anti_targets)
        log_preds = log_preds * asymmetric_w

        if self.eps > 0:  # label smoothing
            self.targets_classes = self.targets_classes.mul(1 - self.eps).add(self.eps / num_classes)

        # loss calculation
        loss = - self.targets_classes.mul(log_preds)

        loss = loss.sum(dim=-1)
        if self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [26]:
model = BERTClassifier(dr_rate = args.drop_rate).to(args.device)

loss_fn_ASL = {'1': ASLSingleLabel(),
           '2': ASLSingleLabel(),
           '3': ASLSingleLabel(),
           '4': ASLSingleLabel()
           }

loss_fn = {'1': FocalLoss(),
           '2': FocalLoss(),
           '3': FocalLoss(),
           '4': FocalLoss()
           }

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=args.lr)

t_total = len(train_dataloader) * args.epoch_n
warmup_step = int(t_total * args.warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

print(warmup_step)
print(t_total)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


309
1548


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


---
# 8. Train
----

In [27]:
for batch, (input_ids, token_type_ids, attention_mask, label_1, label_2, label_3, label_4) in enumerate(tqdm(train_dataloader)):
  print(input_ids.shape)
  print(token_type_ids.shape)
  print(attention_mask.shape)
  print(label_1.shape)
  print(label_2.shape)
  print(label_3.shape)
  print(label_4.shape)
  break

  0%|          | 0/516 [00:00<?, ?it/s]

torch.Size([32, 134])
torch.Size([32, 134])
torch.Size([32, 134])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])


In [28]:
value = 0.0
for epoch in range(args.epoch_n):
  
  model.train()
  train_acc = 0
  train_loss = 0
  for batch, (input_ids, token_type_ids, attention_mask, label_1, label_2, label_3, label_4) in enumerate(tqdm(train_dataloader)):
    optimizer.zero_grad()

    input_ids = input_ids.long().to(args.device)
    token_type_ids = token_type_ids.long().to(args.device)
    attention_mask = attention_mask.to(args.device)

    label_1 = label_1.type(torch.LongTensor).to(args.device)
    label_2 = label_2.type(torch.LongTensor).to(args.device)
    label_3 = label_3.type(torch.LongTensor).to(args.device)
    label_4 = label_4.type(torch.LongTensor).to(args.device)

    # Model train
    label_1_out, label_2_out, label_3_out ,label_4_out = model(input_ids, token_type_ids, attention_mask)
    
    # Loss 
    ##############################################################################################################
    total_loss = (loss_fn_ASL['1'](label_1_out, label_1) 
                  + loss_fn_ASL['2'](label_2_out, label_2) 
                  + loss_fn_ASL['3'](label_3_out, label_3) 
                  + loss_fn_ASL['4'](label_4_out, label_4))
    
    total_loss.backward()
    train_loss += total_loss.item()

    ##############################################################################################################

    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

    optimizer.step()
    scheduler.step()  # Update learning rate schedule    

    # Accuracy
    ##############################################################################################################
    label_1_pred = label_1_out.argmax(1).cpu()
    label_2_pred = label_2_out.argmax(1).cpu()
    label_3_pred = label_3_out.argmax(1).cpu()
    label_4_pred = label_4_out.argmax(1).cpu()

    label_1_acc = np.equal(label_1_pred, label_1.cpu()).sum()
    label_2_acc = np.equal(label_2_pred, label_2.cpu()).sum()
    label_3_acc = np.equal(label_3_pred, label_3.cpu()).sum()
    label_4_acc = np.equal(label_4_pred, label_4.cpu()).sum()

    train_acc += (label_1_acc + label_2_acc + label_3_acc + label_4_acc) / 4

    Accuracy = train_acc / len(train_dataloader.dataset)
    ##############################################################################################################
  print(f'epoch : {epoch+1} Train_loss : {train_loss / len(train_dataloader.dataset):.4f} Train_Acc : {Accuracy :.4f}')
  if Accuracy > value:
    torch.save(model.state_dict(), args.save_path + '/save_model_ASL.pt')
    value = Accuracy
    print(f'{Accuracy:.4f}값의 파일을 저장했습니다')


  0%|          | 0/516 [00:00<?, ?it/s]

epoch : 1 Train_loss : 0.0907 Train_Acc : 0.7864
0.7864값의 파일을 저장했습니다


  0%|          | 0/516 [00:00<?, ?it/s]

epoch : 2 Train_loss : 0.0825 Train_Acc : 0.8412
0.8412값의 파일을 저장했습니다


  0%|          | 0/516 [00:00<?, ?it/s]

epoch : 3 Train_loss : 0.0809 Train_Acc : 0.8560
0.8560값의 파일을 저장했습니다


----
# 9. Inference
---

In [27]:
model.load_state_dict(torch.load(args.save_path + '/save_model_focal.pt'))

<All keys matched successfully>

In [28]:
test = pd.read_csv('/content/drive/MyDrive/2. Study/DACON/data/test.csv')
test['문장'] = test['문장'].apply(lambda x : extract_word(x))
test

,ID,문장
0,TEST_0000,장욱진의 가족 은 허물 없는 가족애를 처음 공개되는 정약용의 정효자전 과 정부인전 ...
1,TEST_0001,조지 W 부시 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다.
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 독소 조항 으로 해석돼 수정을 요청...
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...
...,...,...
7085,TEST_7085,2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표 사회 ...
7086,TEST_7086,탈세계화 징후들이 반갑지 않은 이유다.
7087,TEST_7087,틱톡은 6월 인터넷 안전의 달 을 맞아 올바른 개인정보 보호 관리 방법 앱 내 유용...
7088,TEST_7088,만약 3개월 간 채굴자들의 투표를 거쳐 2 3 이상의 해시파워가 채굴세 도입에 찬성...


In [29]:
class inference_dataset(Dataset):
  def __init__(self, data, max_len):
    self.data = data
    self.max_len = max_len
    self.tokenizer = AutoTokenizer.from_pretrained(args.model)

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    
    data = self.data.loc[idx]
    input = data['문장']
    
    inputs = self.tokenizer(
        input, return_tensors = 'pt',
        truncation = True, max_length = self.max_len,
        padding = 'max_length', add_special_tokens = True
                            )

    input_ids = inputs['input_ids'].reshape(-1)
    token_type_ids = inputs['token_type_ids'].reshape(-1)
    attention_mask = inputs['attention_mask'].reshape(-1)

    return input_ids, token_type_ids, attention_mask

In [30]:
data_test = inference_dataset(test, args.max_length)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=False, 
                                               batch_size=args.batch_size, 
                                               num_workers=args.num_workers, drop_last=False)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [31]:
value_label_1 = []
value_label_2 = []
value_label_3 = []
value_label_4 = []

model.eval()
with torch.no_grad():

  for batch, (input_ids ,token_type_ids, attention_mask) in enumerate(tqdm(test_dataloader)):
    input_ids = input_ids.long().to(args.device)
    token_type_ids = token_type_ids.long().to(args.device)
    attention_mask = attention_mask.to(args.device)

    label_1_out, label_2_out, label_3_out ,label_4_out = model(input_ids, token_type_ids, attention_mask)
    
    value_label_1 += label_1_out.argmax(1).detach().cpu().numpy().tolist()
    value_label_2 += label_2_out.argmax(1).detach().cpu().numpy().tolist()
    value_label_3 += label_3_out.argmax(1).detach().cpu().numpy().tolist()
    value_label_4 += label_4_out.argmax(1).detach().cpu().numpy().tolist()

  0%|          | 0/222 [00:00<?, ?it/s]

In [35]:
test['유형'] = value_label_1
test['극성'] = value_label_2
test['시제'] = value_label_3
test['확실성'] = value_label_4

In [36]:
def get_key(val, my_dict):
    for key, value in my_dict.items():
         if val == value:
             return key

In [37]:
test['유형'] = test['유형'].apply(lambda x : get_key(x, trans['유형']))
test['극성'] = test['극성'].apply(lambda x : get_key(x, trans['극성']))
test['시제'] = test['시제'].apply(lambda x : get_key(x, trans['시제']))
test['확실성'] = test['확실성'].apply(lambda x : get_key(x, trans['확실성']))

In [38]:
test['label'] = test['유형'] + '-' + test['극성'] + '-' + test['시제'] + '-' + test['확실성']
test

,ID,문장,유형,극성,시제,확실성,label
0,TEST_0000,장욱진의 가족 은 허물 없는 가족애를 처음 공개되는 정약용의 정효자전 과 정부인전 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
1,TEST_0001,조지 W 부시 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다.,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.,사실형,긍정,과거,확실,사실형-긍정-과거-확실
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 독소 조항 으로 해석돼 수정을 요청...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
...,...,...,...,...,...,...,...
7085,TEST_7085,2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표 사회 ...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
7086,TEST_7086,탈세계화 징후들이 반갑지 않은 이유다.,사실형,긍정,과거,확실,사실형-긍정-과거-확실
7087,TEST_7087,틱톡은 6월 인터넷 안전의 달 을 맞아 올바른 개인정보 보호 관리 방법 앱 내 유용...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
7088,TEST_7088,만약 3개월 간 채굴자들의 투표를 거쳐 2 3 이상의 해시파워가 채굴세 도입에 찬성...,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [39]:
import os

sub = pd.read_csv(args.submission)
sub['label'] = test['label']
sub.to_csv(args.save_path + '/제출_ASL.csv', index=False, mode='w')    